In [ ]:
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
import re
import random
from datetime import datetime
import json
import csv
import glob
from paddleocr import PaddleOCR
from collections import defaultdict
from tqdm import tqdm

In [ ]:
def extract_company(rows_merged):
    # Keyword umum nama PT/Toko
    keywords = ["sdn", "bhd", "trading", "enterprise", "mart", "store", "cafe"]
    
    # Ambil 5 baris teratas saja
    for r in rows_merged[:5]:
        text = r["text"].strip()
        # Skip jika isinya cuma angka/simbol
        if len(text) < 3 or re.fullmatch(r"[0-9\.\-:\s]+", text): 
            continue 
        
        # Jika mengandung keyword atau huruf kapital semua, kemungkinan itu nama toko
        if any(k in text.lower() for k in keywords) or text.isupper():
            return text
            
    return ""

In [ ]:
def clean_company_name(name):
    if name is None:
        return "UNKNOWN"
    
    name = str(name).strip()
    if not name:
        return "UNKNOWN"
    
    name = str(name).upper()
    
    noise_keywords = [
        "THANK YOU", "VISIT AGAIN", "WELCOME",
        "GST", "TAX INVOICE", "RECEIPT"
        ]
    if any(k in name for k in noise_keywords):
        return "UNKNOWN"
    
    if re.match(r'^\d+[,.\s]+[A-Z\s]{5,}', name):
        return "UNKNOWN"
    
    suffix_patterns = [
        r'\bSDN\.?\s*BHD\.?$',
        r'\bENTERPRISE$',
        r'\bTRADING$',
        r'\bRESTAURANT$',
        r'\bSHOP$',
        r'\bSTORE$'
        ]
    for pat in suffix_patterns:
        name = re.sub(pat, '', name).strip()
    
    # Hapus kurung
    name = re.sub(r'\([^)]*$', '', name)
    
    # Hapus registration number
    name = re.sub(r'\(\d+[^)]*\)', '', name)
    
    name = re.sub(r'\s+', ' ', name).strip()

    if len(name) < 3 and not re.search(r'[A-Z]{2,}', name):
        return "UNKNOWN"
    
    return name

In [ ]:
def extract_date(rows_merged):
    # Pola regex tanggal (DD/MM/YYYY, YYYY-MM-DD, dll)
    patterns = [
        r"\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}",
        r"\d{1,2}\s+(?:JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)[a-z]*\s+\d{2,4}"
    ]
    # Gabungkan seluruh teks untuk pencarian regex global
    full_text = " ".join([r['text'] for r in rows_merged])
    
    for pat in patterns:
        m = re.search(pat, full_text, re.IGNORECASE)
        if m: return m.group(0)
    return None

In [ ]:
def extract_total(rows_merged):
    summary = {"total": 0.0}
    # Cari dari bawah ke atas (karena Total biasanya di bawah)
    for r in reversed(rows_merged):
        txt_low = r['text'].lower()
        if "total" in txt_low and "sub" not in txt_low:
            val = extract_money(r['text'])
            if val:
                summary["total"] = val
                break # Ketemu total, berhenti loop
    return summary

In [ ]:
def is_summary_row(text):
    """Cek apakah baris ini adalah Summary (Total/Tax), bukan barang."""
    keywords = ['total', 'cash', 'change', 'tax', 'gst', 'rounding', 'balance', 'visa']
    return any(k in text.lower() for k in keywords)

In [ ]:
NON_ITEM_KEYWORDS = [
    "total", "subtotal", "cash", "change",
    "rounding", "tax", "gst", "service",
    "visa", "master", "balance", "discount",
    "amount", "paid"
]

In [ ]:
def is_non_item_line(text: str) -> bool:
    t = text.lower()
    return any(k in t for k in NON_ITEM_KEYWORDS)

In [ ]:
def extract_prices(text):
    return [float(p) for p in re.findall(r"\d+\.\d{2}", text)]

In [ ]:
def merchant_position_signal(rows_merged, company_raw):
    for idx, r in enumerate(rows_merged[:5]):
        if company_raw and any(w in r["text"] for w in company_raw.split()):
            if idx <= 2:
                return 1.0
            elif idx <= 4:
                return 0.7
    return 0.0


In [ ]:
def alpha_ratio_signal(text):
    if not text:
        return 0.0
    alpha = sum(c.isalpha() for c in text)
    return min(alpha / max(len(text), 1), 1.0)

In [ ]:
MERCHANT_POSITIVE = [
    "SDN", "BHD", "TRADING", "ENTERPRISE",
    "MART", "STORE", "CAFE"
]

MERCHANT_NEGATIVE = [
    "JALAN", "LOT", "NO", "TEL", "FAX", "GST"
]

In [ ]:
def keyword_signal(text):
    t = text.upper()
    score = 0.0

    if any(k in t for k in MERCHANT_POSITIVE):
        score += 1.0
    if any(k in t for k in MERCHANT_NEGATIVE):
        score -= 1.0

    return max(score, 0.0)

In [ ]:
def clean_valid_signal(clean_name):
    if not clean_name:
        return 0.0
    if clean_name == "UNKNOWN":
        return 0.0
    if len(clean_name) < 3:
        return 0.0
    return 1.0

In [ ]:
def compute_merchant_confidence(rows_merged, company_raw, company_clean):
    s_position = merchant_position_signal(rows_merged, company_raw)
    s_alpha = alpha_ratio_signal(company_raw)
    s_keyword = keyword_signal(company_raw)
    s_clean = clean_valid_signal(company_clean)

    score = (
        0.35 * s_position +
        0.25 * s_alpha +
        0.25 * s_keyword +
        0.15 * s_clean
    )

    return {
        "score": round(score, 3),
        "signals": {
            "position": s_position,
            "alpha_ratio": round(s_alpha, 3),
            "keyword": s_keyword,
            "clean_valid": s_clean
        }
    }

In [ ]:
def date_regex_signal(date_raw):
    return 1.0 if date_raw else 0.0

In [ ]:
def date_parse_signal(date_clean):
    return 1.0 if pd.notna(date_clean) else 0.0

In [ ]:
def date_range_signal(date_clean):
    if pd.isna(date_clean):
        return 0.0
    year = date_clean.year
    current_year = datetime.now().year
    return 1.0 if 2000 <= year <= current_year else 0.0

In [ ]:
def compute_date_confidence(date_raw, date_clean):
    s_regex = date_regex_signal(date_raw)
    s_parse = date_parse_signal(date_clean)
    s_range = date_range_signal(date_clean)

    score = (
        0.5 * s_regex +
        0.3 * s_parse +
        0.2 * s_range
    )

    return {
        "score": round(score, 3),
        "signals": {
            "regex_found": s_regex,
            "parse_valid": s_parse,
            "reasonable_range": s_range
        }
    }

In [ ]:
def item_alpha_ratio_signal(desc):
    alpha = sum(c.isalpha() for c in desc)
    return min(alpha / max(len(desc), 1), 1.0)

In [ ]:
def price_consistency_signal(qty, unit_price, line_total):
    if qty <= 0 or unit_price <= 0 or line_total <= 0:
        return 0.0

    expected = qty * unit_price
    diff = abs(expected - line_total)

    if diff < 0.01:
        return 1.0
    elif diff < 0.1:
        return 0.5
    else:
        return 0.0

In [ ]:
def item_position_signal(row_idx, total_rows):
    ratio = row_idx / max(total_rows - 1, 1)

    if 0.2 <= ratio <= 0.8:
        return 1.0
    elif 0.1 <= ratio <= 0.9:
        return 0.5
    else:
        return 0.0

In [ ]:
def item_length_signal(desc):
    if len(desc) >= 5:
        return 1.0
    elif len(desc) >= 3:
        return 0.5
    else:
        return 0.0

In [ ]:
ITEM_BLACKLIST_KEYWORDS = [
    # Header / table
    "item", "qty", "quantity", "price", "s/price", "unit price",
    
    # Summary / payment
    "total", "subtotal", "tax", "gst", "rounding",
    "payment", "paid", "cash", "change", "balance",
    
    # Footer
    "thank", "served", "receipt", "welcome",

    # Noise
    "amount", "visa", "master", "debit", "credit"
]


In [ ]:
def blacklist_keyword_penalty(desc: str):
    if not desc:
        return 0.0

    t = desc.lower()
    for kw in ITEM_BLACKLIST_KEYWORDS:
        if kw in t:
            return 0.0
    return 1.0

In [ ]:
def word_count_signal(desc):
    words = [w for w in desc.split() if w.isalpha()]
    if len(words) >= 2:
        return 1.0
    elif len(words) == 1:
        return 0.5
    return 0.0

In [ ]:
def digit_ratio_penalty(desc):
    digits = sum(c.isdigit() for c in desc)
    ratio = digits / max(len(desc), 1)

    if ratio < 0.2:
        return 1.0
    elif ratio < 0.4:
        return 0.5
    else:
        return 0.0

In [ ]:
def symbol_noise_penalty(desc):
    symbols = sum(not c.isalnum() and not c.isspace() for c in desc)
    ratio = symbols / max(len(desc), 1)

    if ratio < 0.1:
        return 1.0
    elif ratio < 0.2:
        return 0.5
    else:
        return 0.0

In [ ]:
GENERIC_ITEM_WORDS = [
    "item", "product", "goods", "price", "qty", "amount"
]

def generic_word_penalty(desc):
    t = desc.lower()
    words = t.split()

    if all(w in GENERIC_ITEM_WORDS for w in words):
        return 0.0
    return 1.0

In [ ]:
def semantic_quality_signal(desc):
    s_word = word_count_signal(desc)
    s_digit = digit_ratio_penalty(desc)
    s_symbol = symbol_noise_penalty(desc)
    s_generic = generic_word_penalty(desc)

    score = (
        0.35 * s_word +
        0.25 * s_digit +
        0.20 * s_symbol +
        0.20 * s_generic
    )

    return {
        "score": round(score, 3),
        "signals": {
            "word_count": s_word,
            "digit_ratio": s_digit,
            "symbol_noise": s_symbol,
            "generic_word": s_generic
        }
    }

In [ ]:
def compute_item_confidence(
    desc,
    qty,
    unit_price,
    line_total,
    row_idx,
    total_rows,
):
    s_alpha = item_alpha_ratio_signal(desc)
    s_price = price_consistency_signal(qty, unit_price, line_total)
    s_pos = item_position_signal(row_idx, total_rows)
    s_len = item_length_signal(desc)
    s_blacklist = blacklist_keyword_penalty(desc)
    s_semantic = semantic_quality_signal(desc)["score"]

    score = (
        0.20 * s_alpha +
        0.30 * s_price +
        0.15 * s_pos +
        0.10 * s_len +
        0.15 * s_blacklist +
        0.10 * s_semantic
    )

    return {
        "score": round(score, 3),
        "signals": {
            "alpha_ratio": round(s_alpha, 3),
            "price_consistency": s_price,
            "position": s_pos,
            "length": s_len,
            "blacklist_pass": s_blacklist,
            "semantic_quality": s_semantic
        }
    }

In [ ]:
def extract_line_items(rows_merged):
    items = []
    total_rows = len(rows_merged)

    for idx, r in enumerate(rows_merged):
        text = r['text']

        # 1. Skip non-item rows (summary, footer, payment)
        if is_non_item_line(text):
            continue

        # 2. Extract prices
        prices = extract_prices(text)
        if not prices:
            continue

        # 3. Clean description FIRST
        desc = re.sub(r"\d+\.\d{2}", "", text)
        desc = re.sub(r"\b[xX@]\b", "", desc)
        desc = re.sub(r"\s+", " ", desc).strip()

        if len(desc) < 3:
            continue

        alpha_ratio = sum(c.isalpha() for c in desc) / max(len(desc), 1)
        if alpha_ratio < 0.3:
            continue

        # 4. Default price assumption
        qty = 1
        unit_price = prices[-1]
        line_total = prices[-1]

        # 5. Qty inference ONLY if indicator exists
        has_qty_indicator = re.search(r"\b(\d+)\s*[xX@]\s*\d", text)

        if has_qty_indicator and len(prices) >= 2:
            possible_unit = prices[-2]
            possible_total = prices[-1]

            for q in range(2, 10):
                if abs(possible_unit * q - possible_total) < 0.01:
                    qty = q
                    unit_price = possible_unit
                    line_total = possible_total
                    break

        if line_total <= 0:
            continue
        
        confidence = compute_item_confidence(
            desc,
            qty,
            unit_price,
            line_total,
            idx,
            total_rows
        )

        items.append({
            "description": desc,
            "qty": qty,
            "unit_price": round(unit_price, 2),
            "line_total": round(line_total, 2),
            "row_idx": idx,           # penting untuk confidence
            "row_ratio": round(idx / max(total_rows - 1, 1), 3),
            "confidence": confidence
        })

    return items